In [1]:
import morePVs_output as opm
import morePVs as mpv
import os
import en_utilities as um
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.lines as mlines



Plotting NPV - NPV(bau)
-----------------------------------

And also delta(NPV) as a % of NPV (BAU)
 


In [2]:

project='EN1_rerun'
study_name='value11'
base_path='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\studies'
op = opm.Output(base_path = base_path,
                          project = project,
                          study_name = study_name)

short_name = 'value11'

In [3]:
plot_path = os.path.join(op.plot_path,'plots_npv_bau')
if not os.path.exists(plot_path):
    os.makedirs(plot_path)

In [4]:
self = op
df = self.data.copy()
df_in = self.study_parameters.copy()


In [5]:

df = df.merge(df_in,left_index = True, right_index=True)
for c in df.columns:
    if '_x' in c:
        nc = c[:-2]
        df.rename(columns={c:nc}, inplace=True)
df.head()

,scenario_label,arrangement,number_of_households,load_folder,central_battery_kWh,en_opex,en_capex_repayment,pv_capex_repayment,average_hh_bill$,average_hh_solar_bill$,...,load_folder_y,arrangement_y,pv_cap_id,cp,all_residents,parent,network_tariff,en_capex_id,a_term,a_rate
scenario,,,,,,,,,,,,,,,,,,,,,
0,value11_hpc000_000,bau,208.0,sh_t_all_site_A,0.0,0.0,0.0,0.0,1267.028247,0.0,...,sh_t_all_site_A,bau,NaN,EASO_TOU_15pc,EASO_TOU_15pc,TIDNULL,EA310,NaN,5,0.06
1,value11_hpc000_001,bau,208.0,sh_t_all_site_A,0.0,0.0,0.0,0.0,1267.028247,0.0,...,sh_t_all_site_A,bau,NaN,EASO_TOU_15pc,EASO_TOU_15pc,TIDNULL,EA310,NaN,10,0.06
2,value11_hpc000_002,bau,208.0,sh_t_all_site_A,0.0,0.0,0.0,0.0,1267.028247,0.0,...,sh_t_all_site_A,bau,NaN,EASO_TOU_15pc,EASO_TOU_15pc,TIDNULL,EA310,NaN,15,0.06
3,value11_hpc000_003,bau,208.0,sh_t_all_site_A,0.0,0.0,0.0,0.0,1267.028247,0.0,...,sh_t_all_site_A,bau,NaN,EASO_TOU_15pc,EASO_TOU_15pc,TIDNULL,EA310,NaN,20,0.06
4,value11_hpc000_004,bau,208.0,sh_t_all_site_A,0.0,0.0,0.0,0.0,1267.028247,0.0,...,sh_t_all_site_A,bau,NaN,EASO_TOU_15pc,EASO_TOU_15pc,TIDNULL,EA310,NaN,25,0.06


In [6]:
#df = df[df['arrangement'].isin(['bau', 'en', 'en_pv'])]

In [7]:
# Set up constants
sites = ['A','E',  'D','B', 'H', 'I', 'G', 'C',  'J', 'F' ]
values = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
ordered_sites = dict(zip(sites,values))
tag_list = ['a208_f12_cp34',
 'a161_f7_cp37',
 'a138_f9_cp43',
 'a104_f8_cp56',
 'a52_f3_cp26',
 'a48_f4_cp09',
 'a44_f4_cp16',
 'a34_f4_cp32',
 'a26_f4_cp42',
 'a20_f5_cp36']
tag_dict=dict(zip(sites, tag_list))
tag_sorter= dict(zip(tag_list,values))


# Plotting set-up parameters
cmap = mpl.cm.tab10_r
alpha=1
sites_grouped={'all': sites,
              'group1': [s for s in sites if s in ['A','B','C','D','E']],
              'group2': [s for s in sites if s in['F', 'G','H','I','J']]}

a_terms =[a for a in [5,10,15,20,25] if a in df.a_term.tolist()]
dashList = [(2,8),(3,5),(10,0),(3,2,10,2),(5,2,20,2)] 
markers =['+','o','d','s','x']
term_dashes = dict(zip(a_terms,dashList))
term_markers = dict(zip(a_terms,markers))

# get pv sizes for maximum pv systems
maximums={}
pv_ref_file='C:\\Users\\z5044992\\Documents\\MainDATA\\DATA_EN_4\\reference\\capex_pv_lookup.csv'
pv_ref=pd.read_csv(pv_ref_file)
pv_ref = pv_ref.set_index('pv_cap_id')
for i in pv_ref.index:
    if 'max' in i and 'site' in i:
        site = um.find_between(i,'_','_')
        maximums[site] = pv_ref.loc[i,'kW']
en_capexes= [c for c in ['capex_low','capex_med','capex_high']if c in df.en_capex_id.tolist()]
#parents = [p for p in ['EA310_12c_FIT8', 'EA310_12c_FIT12', 'EA310_12c',\
#           'EA310_10c_FIT8', 'EA310_10c_FIT12', 'EA310_10c',\
#         'EA310_8c_FIT8', 'EA310_8c_FIT12','EA310_8c'] if p in df.parent.tolist()]
parents_retail = list(set([p[6:] for p in df.parent.tolist() if p != 'TIDNULL']))

In [8]:
# Organise labels (no longer used for labels- now use site_tags)
df.loc[:,'site'] = df.loc[:,'load_folder'].apply(lambda x : x[-1])
df.loc[:, 'labels'] = df.loc[:,'site']
floors = {'A':12,'E':7, 'D':9,'B':8,'H':3,'I':4,'G':4,'C':4,'J':4,'F':5}
numhouses={'A':208,'E':161, 'D':138,'B':104,'H':52,'I':48,'G':44,'C':34,'J':26,'F':20}
labels ={}
for s, f in floors.items():
    u = numhouses[s]
    labels[s] = s +'('+str(u)+'/'+str(f)+')'
print(labels)
df.loc[:, 'label'] = df.loc[:, 'site'].apply(lambda x: labels[x])




{'A': 'A(208/12)', 'E': 'E(161/7)', 'D': 'D(138/9)', 'B': 'B(104/8)', 'H': 'H(52/3)', 'I': 'I(48/4)', 'G': 'G(44/4)', 'C': 'C(34/4)', 'J': 'J(26/4)', 'F': 'F(20/5)'}


In [10]:
# get kWp / unit , npv $ / unit  npv_bau and % bau for each scenario
df['pv_filename'].fillna('zero',inplace=True)


for s in df.index:
    site = df.loc[s,'load_folder'][-1]
    a_term = df.loc[s,'a_term']
    df.loc[s,'site']=site
    if df.loc[s,'pv_filename'] == 'zero':
        df.loc[s,'kwp_unit'] =0
    elif 'max' in df.loc[s,'pv_filename']:
        df.loc[s,'kwp_unit']= maximums[site]/df.loc[s,'number_of_households']
    else:
        df.loc[s,'kwp_unit'] = float(df.loc[s,'pv_filename'][-9]) + float(df.loc[s,'pv_filename'][-7])/10
    
    # Calculate NPV relative to bau:
    df.loc[s,'npv_unit'] = df.loc[s,'eno$_npv_building_mean']/df.loc[s,'number_of_households']
    bau_scenario = df[(df['site']==site) & (df['arrangement'] == 'bau')& (df['a_term'] == a_term)].index
    df.loc[s,'bau_npv'] = (df.loc[bau_scenario,'total$_building_costs_mean']).values[0]/df.loc[s,'number_of_households']
    df.loc[s,'npv_cf_bau'] = df.loc[s,'npv_unit'] - df.loc[s,'bau_npv']
    df.loc[s,'%_npv_bau'] = df.loc[s,'npv_cf_bau']/ df.loc[s,'bau_npv'] *100
    
    
df['parent_retail']= df['parent'].apply(lambda x: x[6:])
        

In [18]:
# # delta(NPV):
# # PLOTTING EN ONLY FOR DIFFERENT FINANCIAL PARAMETERS
# # SPECIFIC SCENARIOS
# group= 'all' 
# dfg = df.copy().loc[df['arrangement']=='en']
# max_bau = df['npv_cf_bau'].max()
# min_bau = df['npv_cf_bau'].min()
# max_baupc = df['%_npv_bau'].max()
# min_baupc = df['%_npv_bau'].min()

# for parent_retail in [p for p in parents_retail if 'FIT' not in p]:
  
#     title = 'EN: Retail parent tariff = ' + parent_retail                        
#     p_slice = dfg['parent_retail'] == parent_retail
#     dfx=dfg.copy()[p_slice]
#     if len(dfx.index)>0:
#         # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs     
#         dfx['aterm_str'] = dfx['a_term'].apply(lambda x : str(x))
#         dfx['combined'] = dfx['site']+ '_' + dfx['en_capex_id'] +  '_'+(dfx['aterm_str'])

#         dfx = dfx.drop_duplicates('combined')
#         dfx['new_label']= dfx['site']+ '_' + dfx['en_capex_id']

#         # ----------------------
#         # reindex and stack data
#         # ----------------------
#         dfx.index = [dfx.new_label,dfx.a_term]
#         dfu = dfx['npv_cf_bau'].unstack()

# #         cols =dfu.columns
# #         cols = [c[3:] for c in cols ]
# #         cols = [0.0]+[float(c[3:]) for c in cols if 'pv' in c]
# #         dfu.columns=cols
#         dfv=dfu.transpose()

#         fig, ax = plt.subplots()
#         for capex in en_capexes:
            
#             for label in [c for c in dfv.columns if capex in c]:       
#                 y = dfv[label].dropna()
#                 x = y.index

#                 line = ax.plot(x,y,   \
#                                alpha=alpha,  c = cmap(sites.index(label[0])),\
#                                linewidth = 0.5, marker = capex_markers[capex], markersize=2 ,
#                                linestyle='--', dashes=capex_dashes[capex])

#         ax.set_title(title, fontsize=16, y=1.0)
#         ax.set_xlabel("Amortisation Period (years)", fontsize=14)
#         ax.set_ylabel("NPV compared to BAU", fontsize=14)
#         ax.grid(True)
#         ax.set_ylim((min_baupc*0.9,135)) #max_baupc*1.1))
#         ax.set_xlim(1,29)

#         #Legends outside box and resize chart to fit
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

#         # Create Legend for line types
#         a_terms=[5,10,15,20,25]
        
#         lline = {}
#         for c in en_capexes:
#             lline[c] = mlines.Line2D([],[],color='black', marker=capex_markers[c],
#                                   linestyle='--', dashes=capex_dashes[c],
#                                      markersize=3, label=c)
#         handles = [lline[c] for c in capexes]
#         line_legend = ax.legend(handles = handles[::-1], labels = capexes[::-1], loc='upper left', bbox_to_anchor=(1, 0.3),prop={'size': 9})
        


#         # Add the legend manually to the current Axes.
#         ax1 = plt.gca().add_artist(line_legend)

#         # Create legend for colours
#         cline = {}
#         for s in sites_grouped[group]:
#             cline[s] = mlines.Line2D([],[],c = cmap(sites.index(s)) , label=tag_dict[s])
#         handles2 = [cline[s] for s in sites_grouped[group]]
#         ax.legend(handles = handles2, labels = tag_list, loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})
   

#         pname = study_name +'_en_only_' + parent_retail+ '.jpg'
#         group_path = os.path.join(plot_path,group)
#         if not os.path.exists(group_path):
#             os.makedirs(group_path)          
#         plotFile = os.path.join(group_path,pname)
#         plt.savefig(plotFile,dpi=1000)

#         #plt.close()
# print('*** Completed ***')


In [19]:
# # delta(NPV) as % of NPV(BAU)
# # PLOTTING EN ONLY FOR DIFFERENT FINANCIAL PARAMETERS
# # SPECIFIC SCENARIOS
# group= 'all' 
# dfg = df.copy().loc[df['arrangement']=='en']
# max_bau = df['npv_cf_bau'].max()
# min_bau = df['npv_cf_bau'].min()
# max_baupc = df['%_npv_bau'].max()
# min_baupc = df['%_npv_bau'].min()

# for parent_retail in [p for p in parents_retail if 'FIT' not in p]:
  
#     title = 'EN: Retail parent tariff = ' + parent_retail                        
#     p_slice = dfg['parent_retail'] == parent_retail
#     dfx=dfg.copy()[p_slice]
#     if len(dfx.index)>0:
#         # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs     
#         dfx['aterm_str'] = dfx['a_term'].apply(lambda x : str(x))
#         dfx['combined'] = dfx['site']+ '_' + dfx['en_capex_id'] +  '_'+(dfx['aterm_str'])

#         dfx = dfx.drop_duplicates('combined')
#         dfx['new_label']= dfx['site']+ '_' + dfx['en_capex_id']

#         # ----------------------
#         # reindex and stack data
#         # ----------------------
#         dfx.index = [dfx.new_label,dfx.a_term]
#         dfu = dfx['%_npv_bau'].unstack()

# #         cols =dfu.columns
# #         cols = [c[3:] for c in cols ]
# #         cols = [0.0]+[float(c[3:]) for c in cols if 'pv' in c]
# #         dfu.columns=cols
#         dfv=dfu.transpose()

#         fig, ax = plt.subplots()
#         for capex in en_capexes:
            
#             for label in [c for c in dfv.columns if capex in c]:       
#                 y = dfv[label].dropna()
#                 x = y.index

#                 line = ax.plot(x,y,   \
#                                alpha=alpha,  c = cmap(sites.index(label[0])),\
#                                linewidth = 0.5, marker = capex_markers[capex], markersize=2 ,
#                                linestyle='--', dashes=capex_dashes[capex])

#         ax.set_title(title, fontsize=16, y=1.0)
#         ax.set_xlabel("Amortisation Period (years)", fontsize=14)
#         ax.set_ylabel("NPV as % of BAU", fontsize=14)
#         ax.grid(True)
#         ax.set_ylim((min_baupc*0.9,135)) #max_baupc*1.1))
#         ax.set_xlim(1,29)

#         #Legends outside box and resize chart to fit
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

#         # Create Legend for line types
#         a_terms=[5,10,15,20,25]
        
#         lline = {}
#         for c in en_capexes:
#             lline[c] = mlines.Line2D([],[],color='black', marker=capex_markers[c],
#                                   linestyle='--', dashes=capex_dashes[c],
#                                      markersize=3, label=c)
#         handles = [lline[c] for c in capexes]
#         line_legend = ax.legend(handles = handles[::-1], labels = capexes[::-1], loc='upper left', bbox_to_anchor=(1, 0.3),prop={'size': 9})
        


#         # Add the legend manually to the current Axes.
#         ax1 = plt.gca().add_artist(line_legend)

#         # Create legend for colours
#         cline = {}
#         for s in sites_grouped[group]:
#             cline[s] = mlines.Line2D([],[],c = cmap(sites.index(s)) , label=tag_dict[s])
#         handles2 = [cline[s] for s in sites_grouped[group]]
#         ax.legend(handles = handles2, labels = tag_list, loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})
   

#         pname = study_name +'_%bau_en_only_' + parent_retail+ '.jpg'
#         group_path = os.path.join(plot_path,group)
#         if not os.path.exists(group_path):
#             os.makedirs(group_path)          
#         plotFile = os.path.join(group_path,pname)
#         plt.savefig(plotFile,dpi=1000)

#         #plt.close()
# print('*** Completed ***')

In [16]:
max_bau = df['npv_cf_bau'].max()
min_bau = df['npv_cf_bau'].min()
max_baupc = df['%_npv_bau'].max()
min_baupc = df['%_npv_bau'].min()
max_bau, min_bau, max_baupc, min_baupc

(-6236.851846997692,
 -34979.217910731975,
 -380.9650460662867,
 -1763.6861633090248)

### NPV - NPV(bau)

In [24]:
#LINE PLOTS - delta (NPV)
(y_min,y_max) = (min(0,min_bau*0.9/1000),max(0,max_bau*1.1/1000))
for group in ['all', 'group1','group2']:
    dfg = df.copy()[df['site'].isin(sites_grouped[group])]
    
    for parent_retail, capex in [(parent_retail, capex) for parent_retail in parents_retail for capex in en_capexes]:  
    # parent, capex = [(parent, capex) for parent in parents for capex in en_capexes][0]
        title = parent_retail + " / " + capex                        
        p_slice = dfg['parent_retail'] == parent_retail
        c_slice = dfg['en_capex_id'] == capex
        dfx=dfg.copy()[p_slice & c_slice]
        if len(dfx.index)>0:
            # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs
            dfx.loc[:,'kwp_str'] = dfx.loc[:,'kwp_unit'].apply(lambda x : str(round(x,2)))
            dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
            dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kwp_str']+ '_'+(dfx['aterm_str'])
            dfx['arr_pv'] = dfx['arrangement'] + '_'+ dfx['kwp_str']
            dfx = dfx.drop_duplicates('combined')
            #fx['combined'] = dfx['combined'].apply(lambda x: x[1:])
            dfx['new_label']= dfx['label']+ '_'+ dfx['aterm_str']

            # ----------------------
            # reindex and stack data
            # ----------------------
            dfx.index = [dfx.new_label,dfx.arr_pv]
            dfx['npv_cf_bau_000']=dfx['npv_cf_bau']/1000
            dfu = dfx['npv_cf_bau_000'].unstack()

            cols =dfu.columns
            cols = [c[3:] for c in cols ]
            cols = [0.0]+[float(c[3:]) for c in cols if 'pv' in c]
            dfu.columns=cols
            dfv=dfu.transpose()

            fig, ax = plt.subplots()
            for a_term in a_terms:
                a_label = '_'+str(a_term)
                for label in [c for c in dfv.columns if a_label in c]:       
                    y = dfv[label].dropna()
                    x = y.index

                    line = ax.plot(x,y,   \
                                   alpha=alpha,  c = cmap(sites.index(label[0])),\
                                   linewidth = 0.5, marker = term_markers[a_term], markersize=2 ,
                                   linestyle='--', dashes=term_dashes[a_term])

            ax.set_title(title, fontsize=16, y=1.0)
            ax.set_xlabel("PV system (kWp)", fontsize=14)
            ax.set_ylabel("NPV compared to BAU ($1000's / unit)", fontsize=14)
            ax.grid(True)
            ax.set_ylim((y_min,y_max))

            #Legends outside box and resize chart to fit
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Create Legend for line types
            a_terms=[5,10,15,20,25]
            dashList = [(2,8),(3,5),(10,0),(3,2,10,2),(5,2,20,2)] 
            markers =['+','o','d','s','x']
            term_dashes = dict(zip(a_terms,dashList))
            term_markers = dict(zip(a_terms,markers))
            lline = {}
            for a in a_terms:
                lline[a] = mlines.Line2D([],[],color='black', marker=term_markers[a],
                                      linestyle='--', dashes=term_dashes[a],
                                         markersize=3, label=str(a) + " years" )
            handles = [lline[a] for a in a_terms]
            line_legend = ax.legend(handles = handles, loc='upper left', bbox_to_anchor=(1, 0.3),prop={'size': 9})
            # Add the legend manually to the current Axes.
            ax1 = plt.gca().add_artist(line_legend)

            # Create legend for colours
            cline = {}
            for s in sites_grouped[group]:
                cline[s] = mlines.Line2D([],[],c = cmap(sites.index(s)) , label=tag_dict[s])
            handles2 = [cline[s] for s in sites_grouped[group]]
            ax.legend(handles = handles2, loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})


            pname = study_name +'_'+ parent_retail+'_' + capex + '.jpg'
            group_path = os.path.join(plot_path,group)
            if not os.path.exists(group_path):
                os.makedirs(group_path)          
            plotFile = os.path.join(group_path,pname)
            plt.savefig(plotFile,dpi=1000)

            plt.close()
print('*** Completed ***')


*** Completed ***


### delta(NPV) as % of NPV (BAU)

In [20]:
#LINE PLOTS - % NPV BAU
(y_min,y_max) = (min(0,min_baupc*0.9),max(0,max_baupc*1.1))
for group in ['all', 'group1','group2']:
    dfg = df.copy()[df['site'].isin(sites_grouped[group])]
    
    for parent_retail, capex in [(parent_retail, capex) for parent_retail in parents_retail for capex in en_capexes]:  
    # parent, capex = [(parent, capex) for parent in parents for capex in en_capexes][0]
        title = parent_retail + " / " + capex                        
        p_slice = dfg['parent_retail'] == parent_retail
        c_slice = dfg['en_capex_id'] == capex
        dfx=dfg.copy()[p_slice & c_slice]
        if len(dfx.index)>0:
            # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs
            dfx.loc[:,'kwp_str'] = dfx.loc[:,'kwp_unit'].apply(lambda x : str(round(x,2)))
            dfx.loc[:,'aterm_str'] = dfx.loc[:,'a_term'].apply(lambda x : str(round(x,0)))
            dfx['combined'] = dfx['site']+ '_' + dfx['arrangement'] + '_'+ dfx['kwp_str']+ '_'+(dfx['aterm_str'])
            dfx['arr_pv'] = dfx['arrangement'] + '_'+ dfx['kwp_str']
            dfx = dfx.drop_duplicates('combined')
            #fx['combined'] = dfx['combined'].apply(lambda x: x[1:])
            dfx['new_label']= dfx['label']+ '_'+ dfx['aterm_str']

            # ----------------------
            # reindex and stack data
            # ----------------------
            dfx.index = [dfx.new_label,dfx.arr_pv]
            dfu = dfx['%_npv_bau'].unstack()

            cols =dfu.columns
            cols = [c[3:] for c in cols ]
            cols = [0.0]+[float(c[3:]) for c in cols if 'pv' in c]
            dfu.columns=cols
            dfv=dfu.transpose()

            fig, ax = plt.subplots()
            for a_term in a_terms:
                a_label = '_'+str(a_term)
                for label in [c for c in dfv.columns if a_label in c]:       
                    y = dfv[label].dropna()
                    x = y.index

                    line = ax.plot(x,y,   \
                                   alpha=alpha,  c = cmap(sites.index(label[0])),\
                                   linewidth = 0.5, marker = term_markers[a_term], markersize=2 ,
                                   linestyle='--', dashes=term_dashes[a_term])

            ax.set_title(title, fontsize=16, y=1.0)
            ax.set_xlabel("PV system (kWp)", fontsize=14)
            ax.set_ylabel("% increase in NPV", fontsize=14)
            ax.grid(True)
            ax.set_ylim((y_min, y_max))

            #Legends outside box and resize chart to fit
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Create Legend for line types
            a_terms=[5,10,15,20,25]
            dashList = [(2,8),(3,5),(10,0),(3,2,10,2),(5,2,20,2)] 
            markers =['+','o','d','s','x']
            term_dashes = dict(zip(a_terms,dashList))
            term_markers = dict(zip(a_terms,markers))
            lline = {}
            for a in a_terms:
                lline[a] = mlines.Line2D([],[],color='black', marker=term_markers[a],
                                      linestyle='--', dashes=term_dashes[a],
                                         markersize=3, label=str(a) + " years" )
            handles = [lline[a] for a in a_terms]
            line_legend = ax.legend(handles = handles, loc='upper left', bbox_to_anchor=(1, 0.3),prop={'size': 9})
            # Add the legend manually to the current Axes.
            ax1 = plt.gca().add_artist(line_legend)

            # Create legend for colours
            cline = {}
            for s in sites_grouped[group]:
                cline[s] = mlines.Line2D([],[],c = cmap(sites.index(s)) , label=tag_dict[s])
            handles2 = [cline[s] for s in sites_grouped[group]]
            ax.legend(handles = handles2, loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})


            pname = study_name + '_%bau_' +'_'+ parent_retail+'_' + capex + '.jpg'
            group_path = os.path.join(plot_path,group)
            if not os.path.exists(group_path):
                os.makedirs(group_path)          
            plotFile = os.path.join(group_path,pname)
            plt.savefig(plotFile,dpi=1000)

            plt.close()
print('*** Completed ***')

*** Completed ***


EN only:
---------
* 3 capex
* 2 parent tariffs
* 5 a_terms
* Plot cost (as proportion of BAU) vs a_term
* Colour code capex
*2 graphs for parent tariffs


In [23]:
a_terms =[a for a in [5,10,15,20,25] if a in df.a_term.tolist()]
capexes = ['capex_low','capex_med','capex_high']
dashList1 = [(3,5),(3,2,10,2),(10,0)] #(2,8),,(5,2,20,2)] 
markers1 =['+', 'o', 's'] #,'s','x']
capex_dashes = dict(zip(capexes,dashList1))
capex_markers = dict(zip(capexes,markers1))




Haven't updated this section - not used for EN1?

In [ ]:
# PLOTTING % BAU vs cp ratio FOR DIFFERENT FITS
# SPECIFIC SCENARIOS
group= 'all' 
sites = ['I','G','H','F','J']
arrangements=['cp_only','btm_icp','btm_s_c','en_pv']
dfg = df.copy().loc[df['arrangement'].isin(arrangements)]
cmap = mpl.cm.tab10
max_baupc = dfg['%_bau'].max()
min_baupc = dfg['%_bau'].min()
max_baupc, min_baupc
parent_retail_base = ['TOU10']
sizes = [0.5,1.0]#,1.5,2.0]
fits = ['No_FIT','FIT8','FIT12']
fit_names = dict(zip(fits,['No FiT', '8c FiT', '12c FiT']))

# Set up line colours and styles etc

capexes = ['capex_low','capex_med','capex_high']
dashList1 = [(3,5),(3,2,10,2),(10,0)] #(2,8),,(5,2,20,2)] 
markers1 =['+', 'o', 's'] #,'s','x']
fit_dashes = dict(zip(fits,dashList1))
fit_markers = dict(zip(fits,markers1))

# Determine FiT from parent or retail tariff, stepwise for ease of coding:
for i in dfg.index:
    if dfg.loc[i,'arrangement']=='en_pv':
        if 'FIT8' in dfg.loc[i,'parent_retail']:
            dfg.loc[i,'fit'] = 'FIT8'
        elif 'FIT12' in dfg.loc[i,'parent_retail']:
            dfg.loc[i,'fit'] = 'FIT12'
        else:
            dfg.loc[i,'fit'] = 'No_FIT'
    else:
        if 'FIT8' in dfg.loc[i,'all_residents']:
            dfg.loc[i,'fit'] = 'FIT8'
        elif 'FIT12' in dfg.loc[i,'all_residents']:
            dfg.loc[i,'fit'] = 'FIT12'
        else:
            dfg.loc[i,'fit'] = 'No_FIT'

# Slice for en capex
capex = 'capex_med'
dfg = dfg.loc[~((dfg['arrangement']=='en_pv') & (dfg['en_capex_id'] != capex))]
    

# Loop each chart
for kwp in sizes:
    for a_term in a_terms:
# kwp = sizes[0]
# a_term = 20



        title = 'PV system = '+str(kwp)+ 'kWpeak / '+str(a_term) +' years'                     
        dfx=dfg.copy().loc[(dfg['a_term'] == a_term) & (dfg['kwp_unit']==kwp)]
        if len(dfx.index)>0:
            # Get rid of duplicates - assumed irrelevant as only variation is internal tariffs     


            dfx['combined'] = dfx['site']+'_' + dfx['arrangement'] +  '_'+dfx['fit']
            dfx = dfx.drop_duplicates('combined')
            dfx['new_label']= dfx['arrangement']+ '_' + dfx['fit']

            # ----------------------
            # reindex and stack data
            # ----------------------
            dfx.index = [dfx.new_label,dfx.cp_ratio_mean]
            dfu = dfx['%_bau'].unstack()
            dfv=dfu.transpose()

            fig, ax = plt.subplots()

            for label in [c for c in dfv.columns]:     
                arr = [ a for a in arrangements if a in label][0]
                fit = [f for f in fits if f in label][0]
                y = dfv[label].dropna()
                x = y.index

                line = ax.plot(x,y,   \
                               alpha=alpha,  c = cmap(arrangements.index(arr)),\
                               linewidth = 0.5, marker = fit_markers[fit], markersize=2 ,
                               linestyle='--', dashes=fit_dashes[fit])

            ax.set_title(title, fontsize=16, y=1.0)
            ax.set_xlabel("CP Ratio (%)", fontsize=14)
            ax.set_ylabel("Total Costs as % of BAU", fontsize=14)
            ax.grid(True)
            ax.set_ylim((70,130)) #max_baupc*1.1))


            #Legends outside box and resize chart to fit
            box = ax.get_position()
            ax.set_position([box.x0, box.y0, box.width * 0.8, box.height])

            # Create Legend for line types 
            lline = {}
            for f in fits:
                lline[f] = mlines.Line2D([],[],color='black', marker=fit_markers[f],
                                      linestyle='--', dashes=fit_dashes[f],
                                         markersize=3, label=fit_names[f])
            handles = [lline[f] for f in fits]
            line_legend = ax.legend(handles = handles[::-1],  loc='upper left', bbox_to_anchor=(1, 0.3),prop={'size': 9})



            # Add the legend manually to the current Axes.
            ax1 = plt.gca().add_artist(line_legend)

            # Create legend for colours
            cline = {}
            for a in arrangements:
                cline[a] = mlines.Line2D([],[],c = cmap(arrangements.index(a)) , label=a)
            handles2 = [cline[a] for a in arrangements]
            ax.legend(handles = handles2, labels = arrangements, loc='upper left', bbox_to_anchor=(1, 1),prop={'size': 9})


        pname = study_name +'_bau_vs_cp_' + str(kwp)+ '_'+str(a_term)+ '.jpg'
        group_path = os.path.join(plot_path,group)
        if not os.path.exists(group_path):
            os.makedirs(group_path)          
        plotFile = os.path.join(group_path,pname)
        plt.savefig(plotFile,dpi=1000)

        plt.close()
print('*** Completed ***')


In [ ]:
dfv.columns

In [ ]:
sites


In [ ]:
cmap

In [ ]:
cmap(1)
